In [11]:
!pip -q install diffusers transformers accelerate torch gradio pillow imageio imageio-ffmpeg

In [12]:
import torch
import random
import gradio as gr
import imageio
from diffusers import StableDiffusionPipeline
from PIL import Image

In [13]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

Using device: cuda


In [14]:
model_id = "runwayml/stable-diffusion-v1-5"

pipe = StableDiffusionPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16 if device == "cuda" else torch.float32
)

pipe = pipe.to(device)
pipe.enable_attention_slicing()
pipe.enable_vae_slicing()

history_gallery = []

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/396 [00:00<?, ?it/s]

StableDiffusionSafetyChecker LOAD REPORT from: /root/.cache/huggingface/hub/models--runwayml--stable-diffusion-v1-5/snapshots/451f4fe16113bff5a5d2269ed5ad43b0592e9a14/safety_checker
Key                                               | Status     |  | 
--------------------------------------------------+------------+--+-
vision_model.vision_model.embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Loading weights:   0%|          | 0/196 [00:00<?, ?it/s]

CLIPTextModel LOAD REPORT from: /root/.cache/huggingface/hub/models--runwayml--stable-diffusion-v1-5/snapshots/451f4fe16113bff5a5d2269ed5ad43b0592e9a14/text_encoder
Key                                | Status     |  | 
-----------------------------------+------------+--+-
text_model.embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
/usr/local/lib/python3.12/dist-packages/diffusers/pipelines/pipeline_utils.py:2186: FutureWarning: `enable_vae_slicing` is deprecated and will be removed in version 0.40.0. Calling `enable_vae_slicing()` on a `StableDiffusionPipeline` is deprecated and this method will be removed in a future version. Please use `pipe.vae.enable_slicing()`.
  deprecate(


In [15]:
STYLE_PRESETS = {
    "None": "",
    "Realistic": "ultra realistic, detailed, 4k, cinematic lighting",
    "Anime": "anime style, studio ghibli, highly detailed",
    "Cyberpunk": "cyberpunk style, neon lights, futuristic",
    "Fantasy": "fantasy art, epic, highly detailed",
    "Photorealistic": "photorealistic, DSLR, sharp focus"
}

In [16]:
def enhance_prompt(prompt, style):
    style_text = STYLE_PRESETS.get(style, "")

    enhanced = f"{prompt}, {style_text}, masterpiece, best quality"
    return enhanced.strip(", ")

In [17]:
def generate_image(
    prompt,
    negative_prompt,
    style,
    steps,
    guidance,
    seed,
    width,
    height,
    turbo
):
    global history_gallery

    # Enhance prompt
    prompt = enhance_prompt(prompt, style)

    # Turbo mode
    if turbo:
        steps = min(steps, 20)
        guidance = min(guidance, 6.5)

    # Seed control
    if seed == -1:
        seed = random.randint(0, 999999)
    generator = torch.manual_seed(seed)

    image = pipe(
        prompt=prompt,
        negative_prompt=negative_prompt,
        num_inference_steps=int(steps),
        guidance_scale=float(guidance),
        width=int(width),
        height=int(height),
        generator=generator
    ).images[0]

    history_gallery.insert(0, image)
    history_gallery = history_gallery[:8]

    return image, history_gallery

In [18]:
def generate_video(prompt):
    frames = []

    for i in range(8):
        img = pipe(
            prompt=f"{prompt}, frame {i}, cinematic",
            num_inference_steps=20,
            guidance_scale=6.5
        ).images[0]
        frames.append(img)

    video_path = "generated_video.mp4"

    imageio.mimsave(
        video_path,
        [frame for frame in frames],
        fps=4
    )

    return video_path

In [21]:
css = """
/* ===== GLOBAL ===== */
body {
    background: linear-gradient(135deg, #0b0f1a 0%, #111827 100%);
}

/* Container */
.gradio-container {
    background: transparent !important;
    font-family: 'Inter', sans-serif;
}

/* Hero title */
.hero-title {
    font-size: 42px;
    font-weight: 800;
    background: linear-gradient(90deg, #ff0844, #ffb199);
    -webkit-background-clip: text;
    -webkit-text-fill-color: transparent;
    text-align: center;
    margin-bottom: 10px;
}

.hero-sub {
    text-align: center;
    color: #9ca3af;
    margin-bottom: 30px;
}

/* Glass cards */
.glass {
    background: rgba(255,255,255,0.05) !important;
    backdrop-filter: blur(14px);
    border-radius: 18px !important;
    border: 1px solid rgba(255,255,255,0.08);
    padding: 18px !important;
}

/* Buttons */
button {
    border-radius: 12px !important;
    font-weight: 600 !important;
    transition: all 0.25s ease !important;
}

button:hover {
    transform: translateY(-1px);
    box-shadow: 0 10px 25px rgba(255, 0, 100, 0.35);
}

/* Image frame */
img {
    border-radius: 14px !important;
}

/* Tabs */
.tab-nav button {
    font-weight: 600 !important;
}
"""

with gr.Blocks(css=css, theme=gr.themes.Base()) as demo:

    # ===== HERO =====
    gr.HTML(
        """
        <div class="hero-title">🎬PROgen</div>
        <div class="hero-sub">
        Premium AI Image & Video Generation Dashboard
        </div>
        """
    )

    with gr.Tabs():

        # ================= IMAGE TAB =================
        with gr.Tab("🖼️ Image Studio"):

            with gr.Row():

                # ===== LEFT PANEL =====
                with gr.Column(scale=1, elem_classes="glass"):

                    prompt = gr.Textbox(
                        label="✨ Prompt",
                        placeholder="A cinematic futuristic city at sunset..."
                    )

                    negative_prompt = gr.Textbox(
                        label="🚫 Negative Prompt",
                        value="blurry, low quality, distorted"
                    )

                    style = gr.Dropdown(
                        choices=list(STYLE_PRESETS.keys()),
                        value="Realistic",
                        label="🎨 Style Preset"
                    )

                    turbo = gr.Checkbox(
                        label="⚡ Turbo Mode (faster)",
                        value=False
                    )

                    steps = gr.Slider(10, 50, value=30, label="Steps")
                    guidance = gr.Slider(1, 15, value=7.5, label="Guidance")

                    seed = gr.Number(value=-1, label="Seed (-1 random)")

                    with gr.Row():
                        width = gr.Dropdown([512, 640, 768], value=512, label="Width")
                        height = gr.Dropdown([512, 640, 768], value=512, label="Height")

                    generate_btn = gr.Button(
                        "🚀 Generate Image",
                        variant="primary"
                    )

                # ===== RIGHT PANEL =====
                with gr.Column(scale=1, elem_classes="glass"):

                    output_image = gr.Image(
                        label="Generated Image",
                        height=420
                    )

                    gallery = gr.Gallery(
                        label="🕘 Recent Generations",
                        columns=4,
                        height=200
                    )

                    download_btn = gr.DownloadButton(
                        label="💾 Download Image",
                        visible=False
                    )

            def after_generate(*args):
                img, hist = generate_image(*args)
                return img, hist, gr.update(value=img, visible=True)

            generate_btn.click(
                fn=after_generate,
                inputs=[
                    prompt,
                    negative_prompt,
                    style,
                    steps,
                    guidance,
                    seed,
                    width,
                    height,
                    turbo
                ],
                outputs=[output_image, gallery, download_btn]
            )

        # ================= VIDEO TAB =================
        with gr.Tab("🎬 Video Studio"):

            with gr.Column(elem_classes="glass"):

                video_prompt = gr.Textbox(
                    label="🎬 Video Prompt",
                    placeholder="A drone shot flying over mountains..."
                )

                video_btn = gr.Button(
                    "Generate Video",
                    variant="primary"
                )

                video_output = gr.Video(label="Generated Video")

                video_btn.click(
                    fn=generate_video,
                    inputs=video_prompt,
                    outputs=video_output
                )

demo.launch(share=True)

/tmp/ipython-input-512420994.py:62: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(css=css, theme=gr.themes.Base()) as demo:
/tmp/ipython-input-512420994.py:62: DeprecationWarning: The 'css' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'css' to Blocks.launch() instead.
  with gr.Blocks(css=css, theme=gr.themes.Base()) as demo:


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f5c9e74bea508339e0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
